***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

# **ETAPA 5 - ANÁLISES AD-HOC**

---
---

<br>

Essa etapa será responsável por responder perguntas pré-selecionadas que são ad-hoc's, utilizando resultado gerado pela estrutura completa das etapas anteriores.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Pergunta de Negócio 01 - **_Como evoluíram as vendas mensais por categoria entre 2022 e 2025?_**

> **CONSULTA:** Agregação mensal (ano/mes) de valor_total por categoria_id/categoria_nome.

> **INSIGHT ESPERADO:** Identificar sazonalidade e categorias com maior crescimento relativo.

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

# Tabela fato_vendas 
fato_vendas = "dataexperts.gold.fato_vendas"
dim_categoria    = "dataexperts.gold.dim_categoria_produto"


# Seleção da categoria e de sua respectiva sk
categoria_produto = (
    spark.table(dim_categoria)
         .select("categoria_nome", "sk_categoria")
         .distinct()
)

# Seleção de vendas totais por categoria em função de ano/mes
vendas = (spark.table(fato_vendas)
                    .groupBy("ano", "mes", "sk_categoria")
                    .agg(sum("valor_total").alias("total_vendas"))  
                )

vendas = (vendas.join(categoria_produto.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria"))

# Separacao por categoria
for categoria in categoria_produto.select("categoria_nome").collect():
    
    vendas_categoria = (
        vendas
            .filter(col("categoria_nome") == categoria.categoria_nome)
            .orderBy("ano", "mes")
            .select("ano", "mes", "total_vendas")
            .collect()
    )
    
    if not vendas_categoria:
        continue

    x = [f"{r.ano}-{str(r.mes).zfill(2)}" for r in vendas_categoria]
    y = [r.total_vendas for r in vendas_categoria]

    # Plot
    plt.figure(figsize=(18, 4))
    plt.plot(x, y, marker="o")
    plt.title(f"Vendas - {categoria.categoria_nome}")
    plt.xlabel("Período (Ano-Mês)")
    plt.ylabel("Total de Vendas")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


### Pergunta de negócio 02 - **_Quais os top 10 produtos por valor_total e sua participação no total?_**  

> **CONSULTA:** Ranking por valor_total com cálculo de participação (%) sobre o total geral.

> **INSIGHT ESPERADO:** Concentração de receita e dependência de poucos produtos.

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

fato_vendas = "dataexperts.gold.fato_vendas"
dim_categoria = "dataexperts.gold.dim_categoria_produto"
dim_produto = "dataexperts.gold.dim_produto"

# Tabelas
produtos = spark.table(dim_produto).select("sk_produto", "produto_id")
categorias = spark.table(dim_categoria).select("sk_categoria", "categoria_nome")

vendas_categoria = spark.table(fato_vendas).groupBy("sk_categoria").agg(sum("valor_total").alias("vendas_categoria"))
vendas_produtos = spark.table(fato_vendas).groupBy("sk_produto", "sk_categoria").agg(sum("valor_total").alias("vendas_produto"))

vendas_totais = vendas_categoria.agg(sum("vendas_categoria")).first()[0]

# Join para saber o id dos produtos e o nome das categorias
vendas_categoria = vendas_categoria.join(categorias.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria")

vendas_produtos = vendas_produtos.join(categorias.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria")
vendas_produtos = vendas_produtos.join(produtos.select("sk_produto", "produto_id"), on="sk_produto").drop("sk_produto")


# VENDAS POR PRODUTO (2022~2025)
produtos_rank = vendas_produtos.withColumn("rank", row_number().over(Window.orderBy(col("vendas_produto").desc())))

top_10 = produtos_rank.filter(col("rank") <= 10)
x = top_10.withColumn("participacao [%]", round((col("vendas_produto") * 100) / vendas_totais, 4) )

print("VENDAS POR PRODUTOS")
print("Número de categorias: ", categorias.count())
print("Número de produtos: ", produtos.count())

y = x.agg(sum("participacao [%]").alias("participacao [%]")).first()["participacao [%]"]
print(f"Top 10 produtos representam: {y}%")

x.show()

# VENDAS POR CATEGORIA (2022~2025)
categorias_rank = (
    vendas_categoria
        .withColumn("rank", row_number().over(Window.orderBy(col("vendas_categoria").desc())))
)

top_10 = categorias_rank.filter(col("rank") <= 10)
x = top_10.withColumn("participacao [%]", round((col("vendas_categoria") * 100) / vendas_totais, 4) )
print("VENDAS POR CATEGORIA")
x.show()




### Pergunta de negócio 03 - **_Quais localidades (UF/cidade) apresentam maior ticket médio?_**

> **CONSULTA:** Média de valor_total por pedido (ou por cliente) agrupada por UF/cidade.

> **INSIGHT ESPERADO:** Potenciais mercados premium e oportunides regionais.

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

dim_localidade = "dataexperts.gold.dim_localidade"
fato_vendas = "dataexperts.gold.fato_vendas"

localidades = spark.table(dim_localidade).select("cidade_venda", "estado_venda", "sk_localidade")

vendas_parciais = (spark.table(fato_vendas)
                    .filter(col("sk_localidade").isNotNull())
                    .groupBy("sk_localidade")
                    .agg(sum("valor_total").alias("total_vendas_localidade"))  
                  )

vendas_parciais = vendas_parciais.join(localidades, on="sk_localidade", how="inner").drop("sk_localidade")

vendas_totais = vendas_parciais.agg(sum("total_vendas_localidade").alias("vendas")).first()["vendas"]

media_vendas = vendas_parciais\
                  .withColumn("media_vendas", round(col("total_vendas_localidade")/vendas_totais,6))\
                  .withColumn("participacao[%]", round((col("total_vendas_localidade")*100)/vendas_totais,6))

maiores_tickets = (
    media_vendas
        .withColumn("rank", row_number().over(Window.orderBy(col("media_vendas").desc())))
)

maiores_tickets_localidade = maiores_tickets.filter(col("rank") <= 20)

#maiores_tickets_localidade.show()

estados = media_vendas.groupBy("estado_venda").agg(sum("participacao[%]"))
estados = estados.withColumn("participacao[%]", round(col("sum(participacao[%])"),6)).drop("sum(participacao[%])")
estados_tickets = estados.withColumn("rank", row_number().over(Window.orderBy(col("participacao[%]").desc())))
display(estados_tickets)

sudeste= (
    estados_tickets
    .filter(col("estado_venda").isin(["SP", "RJ", "MG"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

sul = (
    estados_tickets
    .filter(col("estado_venda").isin(["PR", "SC", "RS"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

centro_oeste = (
    estados_tickets
    .filter(col("estado_venda").isin(["MT","GO","MS","DF"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

norte = (
    estados_tickets
    .filter(col("estado_venda").isin(["AC","AP","AM","PA","RO","RR","TO"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

nordeste = (
    estados_tickets
    .filter(col("estado_venda").isin(["CE","MA","PI","RN","PE","PB","AL","SE","BA"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

print(f"Sudeste: {sudeste}")
print(f"Sul: {sul}")
print(f"Centro-Oeste: {centro_oeste}")
print(f"Norte: {norte}")
print(f"Nordeste: {nordeste}")



### Pergunta de Negócio 04 - **_Como a quantidade média por transação varia por categoria e por ano?_**

> **CONSULTA:** Média de quantidade por categoria_id e por ano(data_id).

> **INSIGHT ESPERADO:** Mudanças de comportamento de compra (packs maiores/menores).

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window



###  Pergunta de Negócio 05 - **_Quais clientes são responsáveis por 80% do faturamento (curva ABC)?_**

> **CONSULTA:** Cálculo de valor_total por cliente, ordenação e cumulativo até 80%. 

> **INSIGHT ESPERADO:** Segmentação de clientes estratégicos para ações de retenção

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round, when
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

# Importação das tabelas
fato_vendas = "dataexperts.gold.fato_vendas"
dim_clientes = "dataexperts.gold.dim_cliente"

vendas = spark.table(fato_vendas).groupBy("sk_cliente").agg(sum("valor_total").alias("valor_total"))
clientes = spark.table(dim_clientes).select("sk_cliente", "nome_cliente")
vendas = vendas.join(clientes.select("sk_cliente", "nome_cliente"), on="sk_cliente", how="inner").drop("sk_cliente")

# Calculos
vendas_totais = vendas.agg(sum("valor_total")).first()[0]

vendas = vendas.withColumn("porcento", (col("valor_total")*100/vendas_totais))
vendas = vendas.orderBy(col("porcento").desc())

window = Window.orderBy(desc("valor_total")).rowsBetween(Window.unboundedPreceding, 0)

vendas = vendas.withColumn("percent_acumulado", sum("porcento").over(window))

vendas = vendas.withColumn("abc", when(col("percent_acumulado")<= 80, "A")\
                                    .when((col("percent_acumulado")>80) & (col("percent_acumulado")<=95), "B")\
                                    .otherwise("C"))

vendas.show()



In [0]:
plt.figure(figsize=(12,6))
plt.scatter(df['nome_cliente'], df['valor_total'], c=colors)
#plt.xlabel('Cliente')
#step = max(1, int(len(df) * 0.1))  # mostra a cada 10% dos dados
#plt.xticks(df['nome_cliente'][::step], df['nome_cliente'][::step], rotation=45)
plt.xticks([])
plt.ylabel('Total Vendas')
plt.title('Curva ABC - Scatter')
plt.show()

In [0]:
%skip
# Grafico de Pareto
fig, ax1 = plt.subplots(figsize=(12,6))
df = vendas.toPandas()

# Dominio, clientes 
ax1.bar(df['nome_cliente'], df['valor_total'], color='skyblue')


# Imagem
ax1.set_ylabel('Total Vendas', color='blue')

# Linha do percentual acumulado
ax2 = ax1.twinx()
ax2.plot(df['nome_cliente'], df['percent_acumulado'], color='red', marker='o', linewidth=1)
ax2.set_ylabel('Percentual Acumulado (%)', color='red')

#Limites ABC
ax2.axhline(80, color='green', linestyle='--', label='A = 80%')
ax2.axhline(95, color='orange', linestyle='--', label='B = 95%')

# Infos
ax2.legend(loc='upper left')
plt.title('Curva ABC de Clientes')
plt.show()

In [0]:
%skip
colors = df['abc'].map({'A':'skyblue','B':'orange','C':'gray'})
plt.figure(figsize=(12,6))
plt.bar(df['nome_cliente'], df['valor_total'], color=colors)
plt.xlabel('Cliente')
plt.ylabel('Total Vendas')
plt.title('Curva ABC - Barras coloridas por Classe')
plt.show()

In [0]:
%skip
plt.figure(figsize=(12,6))
plt.plot(df['nome_cliente'], df['percent_acumulado'], marker='o', color='red')
plt.axhline(80, color='green', linestyle='--')
plt.axhline(95, color='orange', linestyle='--')
plt.xlabel('Cliente')
plt.ylabel('Percentual Acumulado (%)')
plt.title('Curva ABC - Percentual Acumulado')
plt.show()

# **ETAPA 6 - AGENTE DE IA PARA ANÁLISES AD-HOC**

---
---

<br>

Essa etapa será responsável por criar uma versão de um modelo de Agente de IA criado manualmente pela plataforma do databricks, carregá-lo com os dados que tivemos como resultado nas etapas anteriores e então proporcionar outras possíveis análises ad-hoc's que possam surgir.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Criação do modelo do agente:

In [0]:
%skip
# =========================================================
# DATAEXPERTS — CRIAÇÃO DO NL → SQL AGENT (SOMENTE FATO)
# =========================================================

import mlflow
from mlflow.pyfunc import PythonModel
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from mlflow.deployments import get_deploy_client

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
EXPERIMENT_ID = "4074425516892918"
MODEL_NAME = "dataex_sql_agent"
LLM_ENDPOINT = "databricks-llama-4-maverick"

CATALOG = "dataexperts"
SCHEMA = "gold"
FACT_TABLE = "fato_vendas"

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# ---------------------------------------------------------
# ASSINATURA DO MODELO
# ---------------------------------------------------------
input_schema = Schema([
    ColSpec("string", "question")
])

output_schema = Schema([
    ColSpec("string", "sql"),
    ColSpec("string", "answer")
])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

input_example = {
    "question": "Qual foi o faturamento total por mês em 2024?"
}

# ---------------------------------------------------------
# CLIENTE LLM
# ---------------------------------------------------------
client = get_deploy_client("databricks")

# ---------------------------------------------------------
# AGENTE NL → SQL
# ---------------------------------------------------------
class DataExSQLAgent(PythonModel):

    @mlflow.trace
    def predict(self, context, model_input):
        question = model_input["question"]

        system_prompt = f"""
Você é um agente especialista em SQL no Databricks, focado em consultas ad-hoc de análise de dados.
Tabela disponível: {CATALOG}.{SCHEMA}.{FACT_TABLE}
Colunas existentes:
- sk_data
- venda_id
- categoria_id
- quantidade
- preco_lista
- valor_total
- _source_file
- _ingestion_date
- sk_cliente
- sk_produto
- sk_localidade
- ano
- mes

Regras obrigatórias:
- Responda APENAS com a consulta SQL correspondente, sem explicações ou texto adicional.
- Use SOMENTE a tabela acima (não utilize outras tabelas).
- Para faturamento, use SUM(valor_total) como total de vendas.
- Para quantidade total, use SUM(quantidade).
- Para contagem de vendas, use COUNT(venda_id).
- Para preço médio, use AVG(preco_lista).
- Para filtros de data, use as colunas ano e mes (ex: ano = 2024 e mes = 6).
- Comece a consulta sempre com SELECT (comando SQL).
- Use aliases claros (por ex. `f` para fato_vendas).
- Nunca crie colunas ou tabelas que não existem no esquema.
- Não use Markdown nem comentários, retorne apenas o SQL final.
"""


        user_prompt = f"""
Pergunta:
{question}

Retorne apenas o SQL.
"""

        response = client.predict(
            endpoint=LLM_ENDPOINT,
            inputs={
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": 0.0
            }
        )

        sql = response["choices"][0]["message"]["content"].strip()

        if context is None:
            return {"sql": sql, "answer": ""}

        df = spark.sql(sql)

        return {
            "sql": sql,
            "answer": df.toPandas().to_json(orient="records")
        }

# ---------------------------------------------------------
# LOG DO MODELO
# ---------------------------------------------------------
with mlflow.start_run(run_name="dataex-sql-agent-fato-only"):
    mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model=DataExSQLAgent(),
        registered_model_name=MODEL_NAME,
        signature=signature,
        input_example=input_example
    )

### Execução do modelo do agente com a pergunta personalizada:

In [0]:
# =========================================================
# DATAEXPERTS — USO DO NL → SQL AGENT (SOMENTE FATO)
# =========================================================

import mlflow

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
MODEL_NAME = "dataex_sql_agent"

# Unity Catalog → SEMPRE usar alias
MODEL_URI = f"models:/{MODEL_NAME}@somente_tabela_fato_venda"

# ---------------------------------------------------------
# CARREGAR O AGENT
# ---------------------------------------------------------
agent = mlflow.pyfunc.load_model(MODEL_URI)

print(f"✅ Agent carregado: {MODEL_URI}")

# ---------------------------------------------------------
# PERGUNTA
# ---------------------------------------------------------
pergunta = ""

resultado = agent.predict({
    "question": pergunta
})

# ---------------------------------------------------------
# RESULTADOS
# ---------------------------------------------------------
print("SQL GERADO:")
print(resultado["sql"])

print("\nRESPOSTA:")
print(resultado["answer"])


### Mostrando resultado da pergunta personalizada:

In [0]:
# =========================================================
# DATAEXPERTS — EXECUÇÃO AUTOMÁTICA DO SQL GERADO PELO AGENT
# =========================================================

sql_gerado = resultado["sql"]

print("🚀 Executando SQL gerado pelo agente:")
print(sql_gerado)

# Executa no Spark
df_resultado = spark.sql(sql_gerado)

# Exibe como tabela no Databricks
display(df_resultado)


In [0]:

# Apenas para testes: 

tb = spark.read.format("delta").load("/Volumes/dataexperts/gold/main/dim_produto")

display(tb)